# Studying a specific cyclone
In this example, we want to study hurricane Wilma (the deepest Atlantic hurricane on record).

In [ ]:
import huracanpy
import matplotlib.pyplot as plt

## Load IBTrACS and subset the specific hurricane
Two subsets of IBTrACS are embedded within HuracanPy: WMO and JTWC.
Default behavior is loading the embedded WMO subset.
You can also retrieve the full and latest IBTrACS files by specifying a different subset.
For more information, see the [huracanpy.load guide](../load.ipynb).

The tracks are loaded as an xarray.Dataset, with one dimension "record" corresponding to each point.
Variables indicate position in space and time, as well as additional attributes such as maximum wind speed and minimum sea-level pressure (SLP).

In [ ]:
# Here we load the WMO subset. This raises a warning that reminds you of the main
# caveats.
ib = huracanpy.load(source="ibtracs")
ib

Wilma corresponds to index 2005289N18282, so we subset this storm. There are two ways of doing this:

1. Use `xarray`'s where

In [ ]:
wilma = ib.where(ib.track_id == "2005289N18282", drop=True)

2. Use huracanpy's sel_id method (more efficient and shorter, but does the same thing)

Note: the `.hrcn` is called an accessor, and allows you to call HuracanPy functions as methods on the xarray objects.

In [ ]:
wilma = ib.hrcn.sel_id("2005289N18282")
# The Wilma object contains only the data for Wilma:
wilma

## Add category info
You can add the Saffir-Simpson and/or the pressure category of Wilma to the tracks (for full list of available info, see [huracanpy.info](../../api/info.rst)).

### Add Saffir-Simpson Category
This is stored in the `saffir_simpson_category` variable

In [ ]:
wilma = wilma.hrcn.add_saffir_simpson_category(wind_name="wind", wind_units="knots")
wilma.saffir_simpson_category

### Add pressure category

In [ ]:
wilma = wilma.hrcn.add_pressure_category(slp_name="slp")
wilma.pressure_category

Note: Most of the accessor methods have a `get_` and an `add_` version.
`get_` returns the values of what you ask for as a DataArray, while `add_` adds it directly to the dataset with a default name.
In the previous case, we could have called get_pressure_category and then save it as a variable, and potentially add it to the dataset separately

In [ ]:
wilma.hrcn.get_pressure_category(slp_name="slp")

## Plot the track and its evolution
### Plot the track on a map, colored by Saffir-Simpson category

In [ ]:
wilma.hrcn.plot_tracks(
    intensity_var_name="saffir_simpson_category", scatter_kws={"palette": "turbo"}
)

### Plot intensity time series using matplotlib

In [ ]:
fig, axs = plt.subplots(2, sharex=True)
axs[0].plot(wilma.time, wilma.wind)
axs[1].plot(wilma.time, wilma.slp)
axs[0].set_ylabel("Wind / kn")
axs[1].set_ylabel("SLP / hPa")

## Calculate properties 

### Duration
Note duration is in hours

In [ ]:
wilma.hrcn.get_track_duration()

### ACE
Accumulated cyclone energy (ACE) is a commonly used measure of cyclone activity that combines the energy and duration of cyclones.

#### Compute ACE for each point

In [ ]:
wilma = wilma.hrcn.add_ace(wind_units="knots")
wilma.ace

#### Plot cumulated ACE

In [ ]:
plt.plot(wilma.time, wilma.ace.cumsum())

### Translation speed
#### Compute translation speed

In [ ]:
wilma = wilma.hrcn.add_translation_speed()

#### Plot translation speed against latitude

In [ ]:
plt.plot(wilma.lat, wilma.translation_speed)
plt.xlabel("Latitude / °")
plt.ylabel("Translation speed / m/s")

### Intensification rate
#### Add intensification rate in wind and pressure
NB: The rates will be in unit/s, where unit is the unit of the variable.

In [ ]:
wilma = wilma.hrcn.add_rate(var_name="wind")
wilma = wilma.hrcn.add_rate(var_name="slp")

#### Plot intensity time series

In [ ]:
fig, axs = plt.subplots(2, sharex=True)
# Convert to kn/h and hPa/h
axs[0].plot(wilma.time, wilma.rate_wind * 3600)
axs[1].plot(wilma.time, wilma.rate_slp * 3600)
axs[0].set_ylabel("kn/h")
axs[1].set_ylabel("hPa/h")